In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import re
import jinja2
from anthropic import AnthropicBedrock
from core import stages
import json


In [78]:
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [79]:
environment = jinja2.Environment()

In [80]:
tsp_tpl = environment.from_string(stages.typespec.PROMPT)
dzl_tpl = environment.from_string(stages.drizzle.PROMPT)
rtr_tpl = environment.from_string(stages.router.PROMPT)
hdl_tpl = environment.from_string(stages.handlers.PROMPT)
pre_tpl = environment.from_string(stages.processors.PROMPT_PRE)

In [81]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [82]:
prompt = tsp_tpl.render(
    application_description=application_description,
)

tsp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
typespec = stages.typespec.parse_output(tsp_response.content[0].text)

In [83]:
typespec

{'reasoning': 'For a gym tracking bot, users would send messages like:\n"I did 3 sets of bench press with 135lbs" or "I completed my chest workout today"\nor "Suggest a 30-minute workout for legs with dumbbells and squat rack"\n\nKey entities needed:\n- Exercise (name, sets, reps, weight)\n- Workout (collection of exercises, duration, target muscle groups)\n- Equipment (to track available gear for suggestions)\n- WorkoutPlan (to suggest routines based on constraints)\n\nKey functions needed:\n- Record completed exercises/workouts\n- Track progress over time\n- Generate workout suggestions based on constraints\n- List available equipment\n\nThe LLM can extract from natural language:\n"I did 3 sets of bench press with 135lbs" -> recordExercise({\n    name: "bench press",\n    sets: 3,\n    weight: 135,\n    unit: "lbs",\n    muscleGroup: "chest"\n})',
 'typespec_definitions': 'model Exercise {\n    name: string\n    sets: int32\n    reps: int32\n    weight?: float32\n    unit?: string\n 

In [84]:
print(typespec["reasoning"])

For a gym tracking bot, users would send messages like:
"I did 3 sets of bench press with 135lbs" or "I completed my chest workout today"
or "Suggest a 30-minute workout for legs with dumbbells and squat rack"

Key entities needed:
- Exercise (name, sets, reps, weight)
- Workout (collection of exercises, duration, target muscle groups)
- Equipment (to track available gear for suggestions)
- WorkoutPlan (to suggest routines based on constraints)

Key functions needed:
- Record completed exercises/workouts
- Track progress over time
- Generate workout suggestions based on constraints
- List available equipment

The LLM can extract from natural language:
"I did 3 sets of bench press with 135lbs" -> recordExercise({
    name: "bench press",
    sets: 3,
    weight: 135,
    unit: "lbs",
    muscleGroup: "chest"
})


In [85]:
print(typespec["typespec_definitions"])

model Exercise {
    name: string
    sets: int32
    reps: int32
    weight?: float32
    unit?: string
    muscleGroup: string
}

model Equipment {
    name: string
    type: string
    available: boolean
}

model WorkoutPlan {
    duration: int32
    targetMuscles: string[]
    exercises: Exercise[]
    requiredEquipment: Equipment[]
}

interface GymBot {
    @llm_func(2)
    recordExercise(exercise: Exercise): void;

    @llm_func(1)
    getProgress(exerciseName: string, from: Date, to: Date): Exercise[];

    @llm_func(2)
    suggestWorkout(duration: int32, targetMuscles: string[], availableEquipment: Equipment[]): WorkoutPlan;

    @llm_func(1)
    updateEquipment(equipment: Equipment[]): void;
}


In [86]:
prompt = dzl_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
)

dzl_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
drizzle = stages.drizzle.parse_output(dzl_response.content[0].text)

In [87]:
print(drizzle["drizzle_schema"])

import { 
  pgTable, 
  text, 
  integer, 
  boolean, 
  real,
  timestamp,
  primaryKey
} from "drizzle-orm/pg-core";

export const exercisesTable = pgTable("exercises", {
  id: integer("id").primaryKey().generatedAlwaysAsIdentity(),
  name: text("name").notNull(),
  sets: integer("sets").notNull(),
  reps: integer("reps").notNull(),
  weight: real("weight"),
  unit: text("unit"),
  muscle_group: text("muscle_group").notNull(),
  created_at: timestamp("created_at").defaultNow().notNull()
});

export const equipmentTable = pgTable("equipment", {
  id: integer("id").primaryKey().generatedAlwaysAsIdentity(),
  name: text("name").notNull(),
  type: text("type").notNull(),
  available: boolean("available").notNull().default(true)
});

export const workoutPlansTable = pgTable("workout_plans", {
  id: integer("id").primaryKey().generatedAlwaysAsIdentity(),
  duration: integer("duration").notNull(),
  created_at: timestamp("created_at").defaultNow().notNull()
});

export const targetMusclesTa

In [114]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = stages.router.TOOLS
)

funcs = stages.router.parse_outputs([content for content in exp_response.content])["user_functions"]


In [113]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise performed by the user, including details like sets, reps, weight, and muscle group. Use when user wants to log or record their exercise activity.',
  'examples': ['I did 3 sets of bench press today with 150 lbs',
   'Just finished 4 sets of squats, 12 reps each',
   'Logged my deadlift session: 5 sets, 5 reps at 225 lbs',
   'I want to record my exercise routine for today',
   'Add my shoulder press workout to my log']},
 {'name': 'getProgress',
  'description': 'Retrieves exercise history and progress for a specific exercise over a given time period. Use when user asks about their history, improvements, or past performance.',
  'examples': ['Show me my bench press progress over the last month',
   'How have my squat numbers improved since January?',
   'What weights was I lifting for deadlifts last week?',
   'Show my exercise history for bicep curls',
   'Check my progress on pull-ups']},
 {'name': 'suggestWorkou

In [89]:
pre_processors = {}
for function_name in typespec["llm_functions"]:
    prompt = pre_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
    )

    pre_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    pre_processor = stages.processors.parse_output(pre_response.content[0].text)
    pre_processors[function_name] = pre_processor

In [90]:
print(pre_response.content[0].text)

<instructions>
The updateEquipment function expects an array of Equipment objects. Each Equipment object must have:
- name: string (required) - name of the equipment
- type: string (required) - category or type of equipment
- available: boolean (required) - availability status

When parsing user input:
1. Look for equipment names and their status
2. Infer equipment type if not explicitly stated
3. Default to false for availability if not specified
4. Convert natural language descriptions into structured equipment array
</instructions>

<examples>
    <example>
        <input>Mark treadmill 2 and rowing machine as out of order</input>
        <output>[
            {
                "name": "treadmill 2",
                "type": "cardio",
                "available": false
            },
            {
                "name": "rowing machine",
                "type": "cardio",
                "available": false
            }
        ]</output>
    </example>

    <example>
        <input>

In [91]:
pre_processors

{'recordExercise': {'instructions': 'The recordExercise function expects an Exercise object with mandatory fields: name, sets, reps, muscleGroup and optional fields: weight, unit.\nRules for processing user input:\n1. Exercise name should be extracted from common exercise terminology\n2. If sets are not specified, default to 1 set\n3. Weight unit defaults to "kg" if not specified\n4. Muscle group should be inferred from the exercise name if not explicitly stated\n5. Numbers in format like "80x6" should be interpreted as weight x reps',
  'examples': [('I did bench press 80kg for 3 sets of 8 reps',
    '{\n    "name": "bench press",\n    "sets": 3,\n    "reps": 8,\n    "weight": 80.0,\n    "unit": "kg",\n    "muscleGroup": "chest"\n}'),
   ('push-ups 3x15',
    '{\n    "name": "push-ups",\n    "sets": 3,\n    "reps": 15,\n    "muscleGroup": "chest"\n}'),
   ('deadlift 225 lbs 5 reps 3 sets',
    '{\n    "name": "deadlift",\n    "sets": 3,\n    "reps": 5,\n    "weight": 225.0,\n    "unit

In [92]:
handlers = {}
for function_name in typespec["llm_functions"]:
    prompt = hdl_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
        drizzle_schema=drizzle["drizzle_schema"],
    )

    hdl_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    handlers[function_name] = stages.handlers.parse_output(hdl_response.content[0].text)

In [93]:
handlers

{'recordExercise': {'handler': 'import { db } from "../db";\nimport { exercisesTable } from \'../db/schema/application\';\n\nconst handle = async (exercise: {\n    name: string;\n    sets: number;\n    reps: number;\n    weight?: number;\n    unit?: string;\n    muscleGroup: string;\n}): Promise<void> => {\n    await db.insert(exercisesTable).values({\n        name: exercise.name,\n        sets: exercise.sets,\n        reps: exercise.reps,\n        weight: exercise.weight || null,\n        unit: exercise.unit || null,\n        muscle_group: exercise.muscleGroup\n    }).execute();\n};'},
 'getProgress': {'handler': 'import { db } from "../db";\nimport { exercisesTable } from \'../db/schema/application\';\nimport { and, between, eq } from \'drizzle-orm\';\n\nconst handle = async (exerciseName: string, from: Date, to: Date): Promise<Exercise[]> => {\n    const exercises = await db\n        .select()\n        .from(exercisesTable)\n        .where(\n            and(\n                eq(exer

In [94]:
with open("core/handler.tpl", "r") as f:
    handler_ts_tpl = environment.from_string(f.read())

In [95]:
hdl_key = "recordExercise"
params = {
    "handler": handlers[hdl_key]["handler"],
    "instructions": pre_processors[hdl_key]["instructions"],
    "examples": pre_processors[hdl_key]["examples"],
}
file_content = handler_ts_tpl.render(**params)

In [96]:
print(file_content)

import { Message } from "../common/handler";
import { client } from "../common/llm";
import { db } from "../db";
import { exercisesTable } from '../db/schema/application';

const handle = async (exercise: {
    name: string;
    sets: number;
    reps: number;
    weight?: number;
    unit?: string;
    muscleGroup: string;
}): Promise<void> => {
    await db.insert(exercisesTable).values({
        name: exercise.name,
        sets: exercise.sets,
        reps: exercise.reps,
        weight: exercise.weight || null,
        unit: exercise.unit || null,
        muscle_group: exercise.muscleGroup
    }).execute();
};


const preProcessorPrompt = `
Examine conversation between user and assistant and extract structured arguments for a function.

<instructions>
The recordExercise function expects an Exercise object with mandatory fields: name, sets, reps, muscleGroup and optional fields: weight, unit.
Rules for processing user input:
1. Exercise name should be extracted from common exercise

In [97]:
print(handlers["suggestWorkout"]["handler"])

import { db } from "../db";
import { 
  workoutPlansTable, 
  targetMusclesTable, 
  workoutPlanExercisesTable, 
  workoutPlanEquipmentTable,
  exercisesTable,
  equipmentTable
} from '../db/schema/application';
import { eq, and, inArray } from 'drizzle-orm';

const handle = async (
  duration: number,
  targetMuscles: string[],
  availableEquipment: Equipment[]
): Promise<WorkoutPlan> => {
  // Create the workout plan
  const [workoutPlan] = await db
    .insert(workoutPlansTable)
    .values({ duration })
    .returning();

  // Insert target muscles
  await db.insert(targetMusclesTable)
    .values(
      targetMuscles.map(muscle => ({
        workout_plan_id: workoutPlan.id,
        muscle: muscle
      }))
    );

  // Get equipment IDs for available equipment
  const equipmentRecords = await db
    .select()
    .from(equipmentTable)
    .where(inArray(equipmentTable.name, availableEquipment.map(e => e.name)));

  // Link equipment to workout plan
  await db.insert(workoutPlanEqu

In [98]:
ts_defs = stages.typespec.parse_output(stages.typespec.PROMPT)

In [99]:
ts_defs

{'reasoning': 'I expect user to send messages like "I ate a burger" or "I had a salad for lunch".\nLLM can extract and infer the arguments from plain text and pass them to the handler\n"I ate a burger" -> recordDish({name: "burger", ingredients: [\n    {name: "bun", calories: 200},\n    {name: "patty", calories: 300},\n    {name: "lettuce", calories: 10},\n    {name: "tomato", calories: 20},\n    {name: "cheese", calories: 50},\n]})\n- recordDish(dish: Dish): void;\n...',
 'typespec_definitions': 'model Dish {\n    name: String\n    ingredients: Ingredient[]\n}\n\nmodel Ingredient {\n    name: String\n    calories: Int\n}\n\ninterface DietBot {\n    @llm_func(1)\n    recordDish(dish: Dish): void;\n    @llm_func(1)\n    listDishes(from: Date, to: Date): Dish[];\n}',
 'llm_functions': ['recordDish', 'listDishes']}

In [100]:
func_names = stages.typespec.extract_llm_func_names(stages.typespec.PROMPT)

In [101]:
func_names

['recordDish', 'listDishes']

# Project Generation

In [102]:
from shutil import copytree, ignore_patterns

In [103]:
def setup_project(workdir: str):
    copytree('templates', workdir, ignore=ignore_patterns('*.pyc', '__pycache__', 'node_modules'))

In [104]:
setup_project('test')

FileExistsError: [Errno 17] File exists: 'test'

In [115]:
from core.stages import router

In [106]:
exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = router.TOOLS
)

funcs = router.parse_outputs([content for content in exp_response.content])["user_functions"]

AttributeError: 'TextBlock' object has no attribute 'content'

In [8]:
funcs

{'functions': [{'name': 'recordDish',
   'description': '\nLog user\'s dish. Examples:\n- "I ate a burger."\n- "I had a salad for lunch."\n- "Chili con carne"\n'},
  {'name': 'listDishes',
   'description': '\nList user\'s dishes. Examples:\n- "What did I eat yesterday?"\n- "Show me my meals for last week."\n'}]}

In [ ]:
prompt = exp_tpl.render(application_description=application_description)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
expansion = stages.expansion.parse_output(exp_response.content[0].text)

In [ ]:
print(expansion["application_specification"])

<types>
        <type>exercise
            - name
            - equipment_required
            - sets
            - reps
            - weight</type>
        <type>equipment
            - name
            - type</type>
        <type>workout_session
            - date
            - duration
            - exercises_performed</type>
        <type>routine
            - exercises
            - duration
            - equipment_needed</type>
    </types>
    <operations>
        <operation>record_workout_session
            - date
            - exercises with sets/reps/weights</operation>
        <operation>track_progress
            - exercise
            - time period</operation>
        <operation>list_available_equipment</operation>
        <operation>generate_routine
            - available equipment
            - time constraint
            - fitness level</operation>
        <operation>view_history
            - time period</operation>
    </operations>
